## [통계적머신러닝] 과제4
통계학과 2019150419 기다연<br>
2021.10.12

In [58]:
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

### 문제1.
iris 데이터에 logistic regression을 적용하되, multi_class = 'ovr'과 'multinomial'을 적용한 후, 추정된 sigmoid 함수와 softmax 함수를 제시하고 해석하라. (intercept와 coefficient 제시 후 해석)

In [59]:
# 데이터 불러오기
iris = sns.load_dataset('iris')
X = iris.drop('species', axis=1)
y = iris['species']

In [60]:
iris.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [61]:
# y data를 범주형으로 변환
classle = LabelEncoder()
y = classle.fit_transform(iris['species'].values) # species 열의 문자열을 categorical 값으로 전환
print(np.unique(y))

[0 1 2]


In [62]:
# 전체 data를 training set과 test set으로 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

In [63]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(105, 4) (45, 4) (105,) (45,)


In [64]:
# 표준화
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

### One-Versus-Rest (OVR)
- OVR(=OVA, 일대다 전략) : One-Versus-Rest, One-Versus-All
- 한 분류에 대한 다른 분류의 이진 분류를 클래스 개수만큼 반복하는 기법
- 이진 분류 = sigmoid 함수 사용

In [65]:
from sklearn.linear_model import LogisticRegression

Logit = LogisticRegression(C=1e2, random_state=1, multi_class='ovr')
ovr_model = Logit.fit(X_train_std, y_train)
y_train_pred = Logit.predict(X_train_std)
y_test_pred = Logit.predict(X_test_std)

In [66]:
print(ovr_model.coef_)

[[-2.27467146  2.04221619 -4.01937844 -3.46372044]
 [-1.2777746  -0.93618002  3.53177076 -2.24098008]
 [-0.31117851 -2.10556199  9.51791722  8.48747802]]


In [67]:
print(ovr_model.intercept_)

[ -5.43052179  -1.03374577 -12.21594409]


In [68]:
X_test.head()

,sepal_length,sepal_width,petal_length,petal_width
148,6.2,3.4,5.4,2.3
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
106,4.9,2.5,4.5,1.7
75,6.6,3.0,4.4,1.4


In [69]:
# sigmoid 함수 생성 위해 변수 numpy array로 변환
x11 = np.array(X_test['sepal_length'].tolist())
x22 = np.array(X_test['sepal_width'].tolist())
x33 = np.array(X_test['petal_length'].tolist())
x44 = np.array(X_test['petal_width'].tolist())

In [70]:
# 회귀 방정식 생성 (intercept, coefficient 사용)
def reg(x1, x2, x3, x4):
    y1 = -5.43052179 -2.27467146*x1 + 2.04221619*x2 -4.01937844*x3 -3.46372044 *x4
    y2 = -1.03374577 -1.2777746*x1 -0.93618002*x2 +3.53177076*x3 -2.24098008*x4
    y3 = -12.21594409 -0.31117851*x1 -2.10556199*x2 +9.517917222*x3 +8.48747802*x4
    return(y1, y2, y3)

In [71]:
logit = reg(x11, x22, x33, x44)
logit

(array([-42.26115038, -17.96753606, -15.61672141, -35.4463992 ,
        -36.85117861, -34.1681627 , -45.11237184, -38.32339286,
        -42.35820999, -16.35978468, -16.82495371, -39.10902027,
        -15.83652317, -28.40276238, -15.6787926 , -39.32230751,
        -43.26027173, -35.32481704, -35.85215605, -41.91744648,
        -46.44218631, -16.19579809, -36.87442413, -39.07287187,
        -34.03892427, -37.45466993, -27.79403356, -45.50126535,
        -18.16536019, -40.31494172, -16.52915934, -16.20346347,
        -34.12690913, -33.82446791, -47.66997746, -44.15203349,
        -15.19793552, -17.80098011, -16.76963728, -38.55841749,
        -45.46767718, -39.45796167, -38.18240243, -16.58215579,
        -16.62098146]),
 array([ 1.77834756, -6.47721243, -5.82233596,  2.44801092,  0.12682104,
        -0.23566595,  4.51522062,  1.51595397,  2.72133361, -7.88992073,
        -8.03498897,  1.63520169, -4.87459302, -0.12780993, -5.41100195,
         1.47121043,  0.10522812,  1.53367471,  0.346

In [72]:
# 클래스에 속할 확률 반환. (시그모이드 함수를 적용해 계산한 확률)
ovr_model.predict_proba(X_test) # 각각 0, 1, 2일 확률임.

array([[2.38647195e-19, 4.61059590e-01, 5.38940410e-01],
       [2.09366689e-07, 2.04373352e-02, 9.79562455e-01],
       [1.63600270e-05, 2.92514728e-01, 7.07468912e-01],
       [2.10101491e-16, 4.79279455e-01, 5.20720545e-01],
       [6.46507104e-17, 3.47114799e-01, 6.52885201e-01],
       [1.00504294e-15, 3.06208244e-01, 6.93791756e-01],
       [1.28609296e-20, 4.97279585e-01, 5.02720415e-01],
       [1.24825818e-17, 4.50531913e-01, 5.49468087e-01],
       [2.07323249e-19, 4.84077022e-01, 5.15922978e-01],
       [3.67627306e-05, 1.75250941e-01, 8.24712296e-01],
       [1.21064808e-05, 7.94896281e-02, 9.20498265e-01],
       [5.63750945e-18, 4.55598913e-01, 5.44401087e-01],
       [7.85178834e-06, 4.49127650e-01, 5.50864498e-01],
       [3.14835768e-13, 3.18843289e-01, 6.81156711e-01],
       [1.70239927e-05, 4.87944663e-01, 5.12038313e-01],
       [4.61405988e-18, 4.48501421e-01, 5.51498579e-01],
       [1.06823719e-19, 3.44813420e-01, 6.55186580e-01],
       [2.50005188e-16, 4.51316

In [75]:
# 분류 예측 (ovr)
ovr_pred = Logit.predict(X_test_std) # target으로는 똑같이 argmax 값을 반환함.
ovr_pred

array([2, 0, 0, 1, 1, 1, 2, 1, 2, 0, 0, 2, 0, 1, 0, 1, 2, 1, 1, 2, 2, 0,
       1, 2, 1, 1, 1, 2, 0, 2, 0, 0, 1, 1, 2, 2, 0, 0, 0, 1, 2, 2, 1, 0,
       0])

### Multinomial
- 로지스틱은 여러개의 이진분류기를 연결하는 것과 더불어, 직접 다중클래스를 지원한다.
- 이를 소프트맥스 회귀/다항 로지스틱 회귀라고 한다.
- 다중 분류 = softmax 함수 사용

#### Process
소프트맥스 회귀모델이 각 클래스 k에 대한 점수를 계산하고, 그 점수에 소프트맥스 함수를 적용해 각 클래스의 확률을 추정한 후 가장 높은 확률 선택.

1) 점수를 계산하는 식은 선형 회귀 예측식과 비슷한데, 각 클래스마다 자신만의 파라미터 벡터가 있고, 샘플 X에 대해 선형으로 결합되어 있음. (회귀랑 똑같음)
2) 이 점수를 소프트맥스 함수에 통과시켜 각 클래스 확률을 추정함.
3) 이러한 추정확률이 가장 높은 클래스를 선택함.

Softmax 함수 : 각 점수에 exp를 취한 후, 정규화함(모든 지수함수 결과의 합으로 나눔)

In [76]:
iris = sns.load_dataset('iris')
X = iris.drop('species', axis=1)
y = iris['species']

classle = LabelEncoder()
y = classle.fit_transform(iris['species'].values) # species 열의 문자열을 categorical 값으로 전환

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [77]:
from sklearn.linear_model import LogisticRegression

Logit = LogisticRegression(C=1e2, random_state=1, multi_class='multinomial')
multi_model = Logit.fit(X_train_std, y_train)
y_train_pred_m = Logit.predict(X_train_std)
y_test_pred_m = Logit.predict(X_test_std)

In [78]:
print(multi_model.coef_)

[[-2.48710159  2.4086907  -5.41565289 -4.83270107]
 [ 1.46232628 -0.12532899 -2.35281482 -2.01137917]
 [ 1.02477531 -2.28336172  7.76846772  6.84408024]]


In [79]:
print(multi_model.intercept_)

[ 0.60193841  6.1076602  -6.70959861]


In [80]:
def score(x1, x2, x3, x4):
    y1 = 0.60193841 -2.48710159*x1 +2.4086907*x2 -5.41565289*x3  -4.83270107*x4
    y2 = 6.1076602 +1.46232628*x1 -0.12532899*x2 -2.35281482*x3 -2.01137917*x4
    y3 = -6.70959861 +1.02477531*x1 -2.28336172*x2 +7.76846772*x3 +6.84408024*x4
    return(y1, y2, y3)

In [81]:
X_test.head(1)

,sepal_length,sepal_width,petal_length,petal_width
148,6.2,3.4,5.4,2.3


In [82]:
score(6.2, 3.4, 5.4, 2.3)

(-46.988281134999994, -2.5834075489999995, 49.571688704)

In [83]:
# 이들을 softmax에 통과시킴. 
np.exp(score(6.2, 3.4, 5.4, 2.3))

array([3.91966354e-21, 7.55162398e-02, 3.37839840e+21])

In [84]:
# softmax 식 직접 구현 
np.exp(score(6.2, 3.4, 5.4, 2.3))/(1.63303538e-06 + 2.05100249e+01 + 2.98564538e+04)

array([1.31193503e-25, 2.52757410e-06, 1.13077032e+17])

In [85]:
multi_model.predict_proba(X_test) 
# 각 클래스에 softmax를 적용한 확률값 도출. 
# 정규화의 효과로 row별 각 클래스의 합은 1이 나옴.

array([[1.16021353e-42, 2.23526749e-23, 1.00000000e+00],
       [7.29204676e-11, 9.45215086e-01, 5.47849140e-02],
       [1.69332265e-09, 9.95231155e-01, 4.76884352e-03],
       [2.56585934e-34, 3.32949170e-18, 1.00000000e+00],
       [8.49582951e-34, 8.07900376e-16, 1.00000000e+00],
       [1.20694137e-30, 5.85780703e-14, 1.00000000e+00],
       [5.16712342e-45, 5.82820301e-24, 1.00000000e+00],
       [6.75149668e-36, 1.80892527e-17, 1.00000000e+00],
       [8.59273074e-42, 5.11908196e-22, 1.00000000e+00],
       [2.62391212e-10, 9.98989713e-01, 1.01028637e-03],
       [1.54189360e-10, 9.97722816e-01, 2.27718423e-03],
       [6.31597022e-38, 1.19175060e-19, 1.00000000e+00],
       [1.39118324e-09, 9.93693766e-01, 6.30623291e-03],
       [4.22884513e-24, 2.48605291e-10, 1.00000000e+00],
       [1.35494720e-09, 9.96965925e-01, 3.03407317e-03],
       [8.43927638e-37, 9.99244579e-18, 1.00000000e+00],
       [1.26859590e-42, 3.31029213e-22, 1.00000000e+00],
       [2.36973232e-33, 7.81956

In [86]:
# 분류 예측 (multinomial)
multi_pred = Logit.predict(X_test_std)
multi_pred

array([2, 0, 0, 2, 1, 1, 2, 1, 2, 0, 0, 2, 0, 1, 0, 1, 2, 1, 1, 2, 2, 0,
       1, 2, 1, 1, 1, 2, 0, 2, 0, 0, 1, 1, 2, 2, 0, 0, 0, 1, 2, 2, 1, 0,
       0])

### 성능비교
- accuracy score
- confusion matrix

In [87]:
print(ovr_pred)
print(multi_pred)

[2 0 0 1 1 1 2 1 2 0 0 2 0 1 0 1 2 1 1 2 2 0 1 2 1 1 1 2 0 2 0 0 1 1 2 2 0
 0 0 1 2 2 1 0 0]
[2 0 0 2 1 1 2 1 2 0 0 2 0 1 0 1 2 1 1 2 2 0 1 2 1 1 1 2 0 2 0 0 1 1 2 2 0
 0 0 1 2 2 1 0 0]


In [93]:
# Accuracy score (정밀도, 전체 표본에서 정확하게 예측을 맞춘 비율)
print('ovr_accuracy: ', str(accuracy_score(y_test, y_test_pred)))
print('multinomial_accuracy: ', str(accuracy_score(y_test, y_test_pred_m)))

ovr_accuracy:  0.9777777777777777
multinomial_accuracy:  1.0


In [94]:
# Confusion matrix
print('ovr_confusion_matrix: \n',str(confusion_matrix(y_test, y_test_pred)))
print('multinomial_confusion_matrix: \n',str(confusion_matrix(y_test, y_test_pred_m)))

ovr_confusion_matrix: 
 [[15  0  0]
 [ 0 15  0]
 [ 0  1 14]]
multinomial_confusion_matrix: 
 [[15  0  0]
 [ 0 15  0]
 [ 0  0 15]]
